## Chapter 8: Number Theory

This chapter uses number theory to investigate more ideas about scientific computation.  We'll solve some problems related to prime numbers and other interesting this in number theory like perfect numbers and happy numbers. This chapter is used as an example of some of the ideas in scientific computing applied to number theory.

### 8.1: Prime Numbers

Recall that a prime number is ....

In [1]:
function findAllFactors(n::Integer) 
  factors = [1]
  for i=2:n-1
    if mod(n,i)==0 
      push!(factors,i)
    end
  end
  push!(factors,n) # n is always a factor of itself 
end

findAllFactors (generic function with 1 method)

This function finds all of the factors of an integer. The result is an array of all of the factors: 

In [2]:
findAllFactors(24)

8-element Array{Int64,1}:
  1
  2
  3
  4
  6
  8
 12
 24

In [3]:
findAllFactors(50)

6-element Array{Int64,1}:
  1
  2
  5
 10
 25
 50

Try to find a function `isPrime` that determines if its input is prime or not:

In [4]:
function isPrime(n::Integer) 
    
end

isPrime (generic function with 1 method)

### 8.2: Finding Primes

It's also helpful to perhaps have a function that produce the next prime given any number:

In [5]:
nextPrime(n::Integer) = isPrime(n+1) ? n+1 : nextPrime(n+1)

nextPrime (generic function with 1 method)

This is recursive.  It first's test if the next integer `n+1` is prime, if so, return `n+1`, else return `nextPrime(n+1)`

Let's test this a bit:

### 8.3 Perfect Numbers

A number is perfect if the sum of its factors (except itself) is itself.  For example, 6 is perfect because it factors are 1,2,3,6 and the sum of the first 3 numbers is 6. We seek to write a function `isPerfect` to test this.

In [6]:
function isPerfect(n::Integer)
  # find the factors of n and check the sum
end

isPerfect (generic function with 1 method)

In [7]:
isPerfect(6)

In [8]:
isPerfect(28)

In [9]:
isPerfect(10)

In [10]:
isPerfect2(n::Integer) = sum(findAllFactors(n)) == 2n

isPerfect2 (generic function with 1 method)

Let's time these two functions to see which is faster.  

In [11]:
@time isPerfect(10_000)

  0.000000 seconds


In [12]:
@time isPerfect2(10_000)

  0.000119 seconds (5 allocations: 640 bytes)


false

Notice that `@time` macro is reporting, we're going to use a package called `BenchmarkTools`.  We just did a tutorial for package managing in julia.  If you need a refresher see Appendix B of the textbook.

In [13]:
using BenchmarkTools

In [14]:
@btime isPerfect(10_000)

  0.001 ns (0 allocations: 0 bytes)


In [15]:
@btime isPerfect2(10_000)

  104.899 μs (5 allocations: 640 bytes)


false

The `@btime` macro runs the command a number of times and reports the average.  This helps in that since other things are happening on your computer, this is more accurate. Also, is using milliseconds (ms), microseconds (μs) and nanoseconds (ns). 

### Happy Numbers

Here’s another fun example. A number n is called happy if you perform the following steps:
1. Take the digits of n and square each one.
2. Sum the squares.
3. If the sum is 1, then the number is happy. If not repeat these steps.

* 13 is happy
* 19 is happy
* 4 is not happy

In [16]:
function isHappy(n::Integer)
  if n==1
    return true
  elseif n==4 
    return false
  else 
    local d = digits(n)
    local sum = 0
    for i=1:length(d)
      sum += d[i]^2
    end
    return isHappy(sum)
  end
end

isHappy (generic function with 1 method)

In [17]:
isHappy(10)

true

In [18]:
isHappy(11)

false

In [19]:
function isHappy2(n::Integer)
  if n==1
    return true
  elseif n==4 
    return false
  else 
    return isHappy2(sum(x->x^2,digits(n)))
  end
end

isHappy2 (generic function with 1 method)

In [20]:
isHappy2(10)

true

In [21]:
isHappy2(11)

false

In [22]:
function isHappy3(n::Integer)
  n == 1 ? true : (n == 4 ? false : isHappy3(sum(x->x^2,digits(n))))
end

isHappy3 (generic function with 1 method)

In [23]:
isHappy3(10)

true

In [24]:
isHappy3(11)

false

You will time this in your homework using the Benchmark tools

Let's find which of the first 100 whole numbers are happy.

In [25]:
[isHappy(n) for n=1:100]

100-element Array{Bool,1}:
 1
 0
 0
 0
 0
 0
 1
 0
 0
 1
 0
 0
 1
 ⋮
 0
 0
 1
 0
 0
 1
 0
 0
 1
 0
 0
 1

Although, you can figured it out a bit (the 1s mean the number is happy, 0 means it's not).  The following is a big easier:

In [26]:
filter(isHappy, 1:100)

20-element Array{Int64,1}:
   1
   7
  10
  13
  19
  23
  28
  31
  32
  44
  49
  68
  70
  79
  82
  86
  91
  94
  97
 100

Remember what just happened here.  The numbers 1 to 100 (as a range) are filtered with only those that are true with the `isHappy` command.

### 8.5: Primes can be big!!

The following is a prime number:

In [ ]:
n = big(2)^89-1

which doesn't fit in `Int64`, which is why we used a `BigInt`.   Don't try to run `isPrime` on this number though. Trust me it won't finish because our algorithm isn't very good.  The largest known prime (as of mid 2020) is 24,862,048 digits long. 

### 8.6 speeding up our algorithms

If we benchmark time our original `isPrime` function on a reasonably sized prime number (but 1 million is tiny in prime number land), we get:

In [27]:
@btime isPrime(1_000_003)

  0.001 ns (0 allocations: 0 bytes)


#### Speeding up `findAllFactors`

Since `isPrime` just uses `findAllFactors`, we should first concentrate on improving `findAllFactors`.  First, we note that we don't need to check all numbers for factors.  For a given number $n$, the biggest factor (other than $n$), can be $n/2$.  Let's stop at this point:

In [28]:
function findAllFactors2(n::Integer)
  factors = [1]
  for i=2:n/2
    if mod(n,i)==0
      push!(factors,i)
    end
  end
  push!(factors,n)
end

findAllFactors2 (generic function with 1 method)

Let's time this versus the previous `findAllFactors`:

In [29]:
@btime findAllFactors(10_000);

  86.299 μs (5 allocations: 640 bytes)


Note that we have supressed the output using the ; because we're more interested in the timing of the function rather than the result.

In [30]:
@btime findAllFactors2(10_000);

  185.500 μs (5 allocations: 640 bytes)


This may be surprising that it is slower.  The reason why is because if we do `n/2`, the result is a Float and all calculations with floating points are always slower than with integers. Let's replace `n/2` with integer division:

In [31]:
function findAllFactors3(n::Integer)
  factors = [1]
  for i=2:div(n,2)
    if mod(n,i)==0
      push!(factors,i)
    end
  end
  push!(factors,n)
end

findAllFactors3 (generic function with 1 method)

In [32]:
@btime findAllFactors3(10_000);

  45.099 μs (5 allocations: 640 bytes)


Note that now we has sped up the result by about a factor of 50%, which is to be expected because we've cut down on number of checks. 

#### keep eliminating factors:

Factors nearly always come in pairs.  Consider all factors of 200:

1    2    4   5   8   10  
200  100  50  40  25  20



In [33]:
function findAllFactors4(n::Integer)
  local x = round(Int,sqrt(n)) # check if this is a perfect square
  local factors = [1,n]
  local j=2 # keep track where to insert elements
  for k=2:x-1
    if n%k==0
      # insert the new factors in the middle of the factors array
      splice!(factors,j:(j-1),[k,div(n,k)])
      j+=1
    end
  end
  if x^2==n
    insert!(factors,j,x)
  end
  factors
end

findAllFactors4 (generic function with 1 method)

In [34]:
findAllFactors4(200)

12-element Array{Int64,1}:
   1
   2
   4
   5
   8
  10
  20
  25
  40
  50
 100
 200

In [35]:
@btime findAllFactors4(10_000);

  2.910 μs (27 allocations: 2.52 KiB)


This shows a speed up of about 12 for this example.  This doesn't scale linearly because we stop at $\sqrt{n}$

Let's now check the various `isPrime` functions

In [36]:
function isPrime(n::Integer)
  return length(findAllFactors(n))==2
end

isPrime (generic function with 1 method)

In [37]:
function isPrime2(n::Integer)
  return length(findAllFactors3(n))==2
end

isPrime2 (generic function with 1 method)

In [38]:
function isPrime3(n::Integer)
  return length(findAllFactors4(n))==2
end

isPrime3 (generic function with 1 method)

In [39]:
@btime isPrime(1_000_003)

  3.594 ms (2 allocations: 144 bytes)


true

In [40]:
@btime isPrime2(1_000_003)

  1.841 ms (2 allocations: 144 bytes)


true

In [41]:
@btime isPrime3(1_000_003)

  11.414 μs (1 allocation: 96 bytes)


true

#### if we are determining primeness, why find all factors

Let's just walk through all integers up to $\sqrt{n}$ and stop if we find a factor

In [42]:
function isPrime4(n::Integer)
  if n==1
    return false
  end
  local x = round(Int,sqrt(n)) # find integer nearest sqrt(n)
  for k=2:x
    if n%k==0
      return false
    end
  end
  true
end

isPrime4 (generic function with 1 method)

Let's check this with a larger prime:

In [43]:
@btime isPrime3(100_000_007)

  52.799 μs (1 allocation: 96 bytes)


true

In [44]:
@btime isPrime4(100_000_007)

  124.299 μs (0 allocations: 0 bytes)


true

These look to be nearly the same.  Note that `isPrime3` uses an array but still is about as fast.

We can cut down a bit more, by only checking odd numbers. 

In [45]:
function isPrime5(n::Integer)
  if n%2==0
    return false
  end
  for k=3:2:round(Int,sqrt(n))
    if n%k==0
      return false
    end
  end
  true
end

isPrime5 (generic function with 1 method)

In [46]:
@btime isPrime5(100_000_007)

  62.199 μs (0 allocations: 0 bytes)


true

In [47]:
function getPrimes(n::Integer) ## return all primes up to n using the sieve of erathones
  local is_prime=trues(n) ## assume all are prime
  local k=2
  while true
    is_prime[2*k:k:n] .= false
    k = findnext(is_prime,k+1) # find the next prime after k
    if isnothing(k)
      return findall(is_prime)[2:end]
    end
  end
end

getPrimes (generic function with 1 method)

#### using the Primes package

In [51]:
using Primes

┌ Info: Precompiling Primes [27ebfcd6-29c5-5fa9-bf4b-fb8fc14df3ae]
└ @ Base loading.jl:1278


In [52]:
@btime isprime(100_000_007)

  1.920 μs (1 allocation: 16 bytes)


true

In [53]:
@btime isprime(big(2)^89-1)

  36.300 μs (18 allocations: 2.73 KiB)


true